In [43]:
import sqlite3
import numpy as np
import pandas as pd
import time
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)

db = sqlite3.connect('L19DB_demo.sqlite')
cursor = db.cursor()
cursor.execute("DROP TABLE IF EXISTS candidates")
cursor.execute("DROP TABLE IF EXISTS contributors")
cursor.execute("PRAGMA foreign_keys=1")

cursor.execute('''CREATE TABLE candidates (
               id INTEGER PRIMARY KEY NOT NULL, 
               first_name TEXT, 
               last_name TEXT, 
               middle_init TEXT, 
               party TEXT NOT NULL)''')

db.commit() # Commit changes to the database

cursor.execute('''CREATE TABLE contributors (
          id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, 
          last_name TEXT, 
          first_name TEXT, 
          middle_name TEXT, 
          street_1 TEXT, 
          street_2 TEXT, 
          city TEXT, 
          state TEXT, 
          zip TEXT, 
          amount REAL, 
          date DATETIME, 
          candidate_id INTEGER NOT NULL, 
          FOREIGN KEY(candidate_id) REFERENCES candidates(id))''')

db.commit()

with open ("candidates.txt") as candidates:
    next(candidates) # jump over the header
    for line in candidates.readlines():
        cid, first_name, last_name, middle_name, party = line.strip().split('|')
        vals_to_insert = (int(cid), first_name, last_name, middle_name, party)
        cursor.execute('''INSERT INTO candidates 
                  (id, first_name, last_name, middle_init, party)
                  VALUES (?, ?, ?, ?, ?)''', vals_to_insert)

with open ("contributors.txt") as contributors:
    next(contributors)
    for line in contributors.readlines():
        cid, last_name, first_name, middle_name, street_1, street_2, \
            city, state, zip_code, amount, date, candidate_id = line.strip().split('|')
        vals_to_insert = (last_name, first_name, middle_name, street_1, street_2, 
                          city, state, int(zip_code), amount, date, candidate_id)
        cursor.execute('''INSERT INTO contributors (last_name, first_name, middle_name, 
                           street_1, street_2, city, state, zip, amount, date, candidate_id) 
                           VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''', vals_to_insert)

candidate_cols = [col[1] for col in cursor.execute("PRAGMA table_info(candidates)")]
contributor_cols = [col[1] for col in cursor.execute("PRAGMA table_info(contributors)")]

def viz_tables(cols, query):
    q = cursor.execute(query).fetchall()
    framelist = []
    for i, col_name in enumerate(cols):
        framelist.append((col_name, [col[i] for col in q]))
    return pd.DataFrame.from_items(framelist)

In [44]:
query='''SELECT contributors.last_name, contributors.first_name, candidates.last_name FROM contributors INNER JOIN candidates ON contributors.candidate_id = candidates.id'''
q=cursor.execute(query).fetchall()
print(q)

[('Agee', 'Steven', 'Huckabee'), ('Ahrens', 'Don', 'Huckabee'), ('Ahrens', 'Don', 'Huckabee'), ('Ahrens', 'Don', 'Huckabee'), ('Akin', 'Charles', 'Huckabee'), ('Akin', 'Mike', 'Huckabee'), ('Akin', 'Rebecca', 'Huckabee'), ('Aldridge', 'Brittni', 'Huckabee'), ('Allen', 'John D.', 'Huckabee'), ('Allen', 'John D.', 'Huckabee'), ('Allison', 'John W.', 'Huckabee'), ('Allison', 'Rebecca', 'Huckabee'), ('Allison', 'Rebecca', 'Huckabee'), ('Altes', 'R.D.', 'Huckabee'), ('Andres', 'Dale', 'Huckabee'), ('Anthony', 'John', 'Huckabee'), ('Arbogast', 'Robert', 'Huckabee'), ('Arbogast', 'Robert', 'Huckabee'), ('Ardle', 'William', 'Huckabee'), ('Atiq', 'Omar', 'Huckabee'), ('Atiq', 'Omar', 'Huckabee'), ('Baker', 'David', 'Huckabee'), ('Bancroft', 'David', 'Huckabee'), ('Banks', 'Charles', 'Huckabee'), ('Barbee', 'John', 'Huckabee'), ('Buckler', 'Steve', 'Obama'), ('Buckler', 'Steve', 'Obama'), ('Buckheit', 'Bruce', 'Obama'), ('Buckel', 'Linda', 'Obama'), ('Buckel', 'Linda', 'Obama'), ('Buckel', 'Lind

In [45]:
query='''SELECT contributors.last_name, contributors.first_name, candidates.last_name FROM contributors INNER JOIN candidates ON contributors.candidate_id = candidates.id WHERE candidates.last_name="Obama"'''
q=cursor.execute(query).fetchall()
print(q)

[('Buckler', 'Steve', 'Obama'), ('Buckler', 'Steve', 'Obama'), ('Buckheit', 'Bruce', 'Obama'), ('Buckel', 'Linda', 'Obama'), ('Buckel', 'Linda', 'Obama'), ('Buckel', 'Linda', 'Obama'), ('Buck', 'Thomas', 'Obama'), ('Buck', 'Jay', 'Obama'), ('Buck', 'Blaine', 'Obama'), ('Buck', 'Barbara', 'Obama'), ('Buck', 'Barbara', 'Obama'), ('Buchman', 'Mark M', 'Obama'), ('Bucher', 'Ida', 'Obama'), ('Buchanek', 'Elizabeth', 'Obama'), ('Buchanan', 'John', 'Obama'), ('Buchanan', 'John', 'Obama'), ('Buchanan', 'John', 'Obama'), ('Buchanan', 'John', 'Obama'), ('Buchanan', 'John', 'Obama'), ('Buchanan', 'John', 'Obama'), ('Buchanan', 'John', 'Obama'), ('Buchanan', 'John', 'Obama'), ('Buchanan', 'John', 'Obama'), ('Buchanan', 'John', 'Obama'), ('Harrison', 'Ryan', 'Obama')]


| nA   | attr  | idA |
| :::: | ::::: | ::: |
| s1   | 23    | 0   |
| s2   | 7     | 2   |
| s3   | 15    | 2   |
| s4   | 31    | 0   |

| nB   | attr  | idB |
| :::: | ::::: | ::: |
| t1   | 60    | 0   |
| t2   | 14    | 7   |
| t3   | 22    | 2   |



| nA   | A.attr|nB  |B.attr|
| :::: | ::::: |::::|::::|
| s1   | 23    |t1  |60|
| s2   | 7     |t3  |22|
| s3   | 15    |t3  |22|
| s4   | 31    |t1  |60|

| nA   | A.attr|nB  |B.attr|
| :::: | ::::: |::::|::::|
| s1   |23    |t1  |60|
|s4|31|t1|60|
|none  | NaN    |t2  |14|
| s2   | 7    |t3  |22|
|s3|15|t3|22|


In [35]:
query='''SELECT AVG(amount),count(last_name) from contributors GROUP BY candidate_id'''
q=cursor.execute(query).fetchall()
pd.DataFrame(q)

,0,1
0,772.0000,25
1,472.4356,25
2,-1360.0000,25
3,-519.7088,25
4,50.6000,25
5,156.0000,25
6,452.6000,25


In [38]:
query = '''SELECT AVG(contributors.amount) AS average_contribution, 
COUNT(*) AS number_of_contributrs,
candidates.last_name AS candidate_last_name FROM contributors 
LEFT JOIN candidates ON candidates.id = contributors.candidate_id 
GROUP BY candidate_last_name'''
viz_tables(["average_contribution","number_of_contributrs","candidate_last_name"], query)

,average_contribution,number_of_contributrs,candidate_last_name
0,50.6000,25,Clinton
1,-1360.0000,25,Giuliani
2,772.0000,25,Huckabee
3,452.6000,25,McCain
4,472.4356,25,Obama
5,-519.7088,25,Paul
6,156.0000,25,Romney


In [39]:
q=cursor.execute(query).fetchall()
print(q)

[(50.6, 25, 'Clinton'), (-1360.0, 25, 'Giuliani'), (772.0, 25, 'Huckabee'), (452.6, 25, 'McCain'), (472.43559999999997, 25, 'Obama'), (-519.7088, 25, 'Paul'), (156.0, 25, 'Romney')]


In [40]:
dfcand = pd.read_csv("candidates.txt", sep="|")
dfcand

,id,first_name,last_name,middle_name,party
0,33,Joseph,Biden,NaN,D
1,36,Samuel,Brownback,NaN,R
2,34,Hillary,Clinton,R.,D
3,39,Christopher,Dodd,J.,D
4,26,John,Edwards,NaN,D
5,22,Rudolph,Giuliani,NaN,R
6,24,Mike,Gravel,NaN,D
7,16,Mike,Huckabee,NaN,R
8,30,Duncan,Hunter,NaN,R
9,31,Dennis,Kucinich,NaN,D


In [41]:
dfcontr = pd.read_csv("contributors.txt", sep="|")
dfcontr

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,NaN,Agee,Steven,NaN,549 Laurel Branch Road,NaN,Floyd,VA,24091,500.0,2007-06-30,16
1,NaN,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,250.0,2007-05-16,16
2,NaN,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,50.0,2007-06-18,16
3,NaN,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,100.0,2007-06-21,16
4,NaN,Akin,Charles,NaN,10187 Sugar Creek Road,NaN,Bentonville,AR,72712,100.0,2007-06-16,16
5,NaN,Akin,Mike,NaN,181 Baywood Lane,NaN,Monticello,AR,71655,1500.0,2007-05-18,16
6,NaN,Akin,Rebecca,NaN,181 Baywood Lane,NaN,Monticello,AR,71655,500.0,2007-05-18,16
7,NaN,Aldridge,Brittni,NaN,"808 Capitol Square Place, SW",NaN,Washington,DC,20024,250.0,2007-06-06,16
8,NaN,Allen,John D.,NaN,1052 Cannon Mill Drive,NaN,North Augusta,SC,29860,1000.0,2007-06-11,16
9,NaN,Allen,John D.,NaN,1052 Cannon Mill Drive,NaN,North Augusta,SC,29860,1300.0,2007-06-29,16


In [42]:
del dfcontr['id']
dfcontr.head()

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,Agee,Steven,NaN,549 Laurel Branch Road,NaN,Floyd,VA,24091,500.0,2007-06-30,16
1,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,250.0,2007-05-16,16
2,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,50.0,2007-06-18,16
3,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,100.0,2007-06-21,16
4,Akin,Charles,NaN,10187 Sugar Creek Road,NaN,Bentonville,AR,72712,100.0,2007-06-16,16


In [46]:
dbp = sqlite3.connect('L19_pandas_DB.sqlite')
csr = dbp.cursor()
csr.execute("DROP TABLE IF EXISTS candidates")
csr.execute("DROP TABLE IF EXISTS contributors")
csr.execute("PRAGMA foreign_keys=1")

csr.execute('''CREATE TABLE candidates (
               id INTEGER PRIMARY KEY NOT NULL, 
               first_name TEXT, 
               last_name TEXT, 
               middle_name TEXT, 
               party TEXT NOT NULL)''')

dbp.commit() # Commit changes to the database

csr.execute('''CREATE TABLE contributors (
          id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, 
          last_name TEXT, 
          first_name TEXT, 
          middle_name TEXT, 
          street_1 TEXT, 
          street_2 TEXT, 
          city TEXT, 
          state TEXT, 
          zip TEXT, 
          amount REAL, 
          date DATETIME, 
          candidate_id INTEGER NOT NULL, 
          FOREIGN KEY(candidate_id) REFERENCES candidates(id))''')

dbp.commit()

In [47]:
dfcand.query("first_name=='Mike' & party=='D'")

,id,first_name,last_name,middle_name,party
6,24,Mike,Gravel,NaN,D


In [48]:
dfcand[dfcand.middle_name.notnull()]

,id,first_name,last_name,middle_name,party
2,34,Hillary,Clinton,R.,D
3,39,Christopher,Dodd,J.,D
16,41,Fred,Thompson,D.,R


In [49]:
dfcand[(dfcand.first_name=="Mike") & (dfcand.party=="D")]

,id,first_name,last_name,middle_name,party
6,24,Mike,Gravel,NaN,D


In [50]:
dfcand[dfcand.first_name.isin(['Mike', 'Hillary'])]

,id,first_name,last_name,middle_name,party
2,34,Hillary,Clinton,R.,D
6,24,Mike,Gravel,NaN,D
7,16,Mike,Huckabee,NaN,R


In [52]:
dfcontr

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,Agee,Steven,NaN,549 Laurel Branch Road,NaN,Floyd,VA,24091,500.0,2007-06-30,16
1,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,250.0,2007-05-16,16
2,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,50.0,2007-06-18,16
3,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,100.0,2007-06-21,16
4,Akin,Charles,NaN,10187 Sugar Creek Road,NaN,Bentonville,AR,72712,100.0,2007-06-16,16
5,Akin,Mike,NaN,181 Baywood Lane,NaN,Monticello,AR,71655,1500.0,2007-05-18,16
6,Akin,Rebecca,NaN,181 Baywood Lane,NaN,Monticello,AR,71655,500.0,2007-05-18,16
7,Aldridge,Brittni,NaN,"808 Capitol Square Place, SW",NaN,Washington,DC,20024,250.0,2007-06-06,16
8,Allen,John D.,NaN,1052 Cannon Mill Drive,NaN,North Augusta,SC,29860,1000.0,2007-06-11,16
9,Allen,John D.,NaN,1052 Cannon Mill Drive,NaN,North Augusta,SC,29860,1300.0,2007-06-29,16


In [51]:
dfcontr.to_sql("contributors", dbp, if_exists="append", index=False)

IntegrityError: FOREIGN KEY constraint failed

In [55]:
0.5*(1+2+0.9*0.315)

1.64175